In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from utils import RandomSeedResetter
from utils.ReservoirComputing import EchoStateNetwork

RandomSeedResetter.reset_random_seed(0)

In [2]:
input_train_shuffle = np.load('data/lorenz/input_train_shuffle.npy')
output_train_shuffle = np.load('data/lorenz/output_train_shuffle.npy')
input_valid = np.load('data/lorenz/input_valid.npy')
output_valid = np.load('data/lorenz/output_valid.npy')
input_test = np.load('data/lorenz/input_test.npy')
output_test = np.load('data/lorenz/output_test.npy')

In [3]:
DIM_INPUT = len(input_train_shuffle[0, 0, :])
LEN_TIME = len(input_train_shuffle[0, :])
SIZE_RESERVOIR =320
DIM_INTER = SIZE_RESERVOIR
DEPTH_ESN = 3
LEAK_RATE = [1, 0.9, 0.8]
SIZE_LEAK = len(LEAK_RATE[:])
PROB_INPUT = 1
PROB_RESERVOIR = 1

In [4]:
deep_esn = [[], [], []]

In [5]:
for idx_leak in range(SIZE_LEAK):
    esn = EchoStateNetwork(
                    SIZE_INPUT=DIM_INPUT,
                    SIZE_RESERVOIR=SIZE_RESERVOIR,
                    PROB_INPUT=PROB_INPUT,
                    PROB_RESERVOIR=PROB_RESERVOIR,
                    LEAK_RATE=LEAK_RATE[idx_leak])
    deep_esn[idx_leak].append(esn)

In [6]:
for idx_depth in range(DEPTH_ESN-1):
    for idx_leak in range(SIZE_LEAK):
        esn = EchoStateNetwork(
            SIZE_INPUT=DIM_INTER,
            SIZE_RESERVOIR=SIZE_RESERVOIR,
            PROB_INPUT=PROB_INPUT,
            PROB_RESERVOIR=PROB_RESERVOIR,
            LEAK_RATE=LEAK_RATE[idx_leak]) 
        deep_esn[idx_leak].append(esn)

In [7]:
time_init = time.time()
SIZE_TRAIN = len(input_train_shuffle[:])
input_esn = np.zeros((DIM_INPUT, 1))
log_deep_esn_train_shuffle = np.zeros((SIZE_LEAK, SIZE_TRAIN, SIZE_RESERVOIR*DEPTH_ESN))

In [ ]:
for idx_batch in range(SIZE_TRAIN):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):    
        input_esn[:, 0] = input_train_shuffle[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            deep_esn[idx_leak][0].activate(input_esn[:, :])
            for idx_depth in range(1, DEPTH_ESN):
                deep_esn[idx_leak][idx_depth].activate( \
                    deep_esn[idx_leak][idx_depth-1].state_reservoir[:])

    for idx_leak in range(SIZE_LEAK):
        for idx_depth in range(DEPTH_ESN):
            log_deep_esn_train_shuffle \
                [idx_leak, idx_batch, idx_depth*SIZE_RESERVOIR:(idx_depth+1)*SIZE_RESERVOIR]  \
                = deep_esn[idx_leak][idx_depth].state_reservoir[:]
            # reset reservoir state among time steps
            deep_esn[idx_leak][idx_depth].reset_reservoir_state()

idx_batch: 0
0.0 min
idx_batch: 500
9.7 min
idx_batch: 1000
19.3 min
idx_batch: 1500
28.9 min
idx_batch: 2000
38.4 min
idx_batch: 2500
48.0 min
idx_batch: 3000
57.5 min
idx_batch: 3500
67.1 min
idx_batch: 4000
76.7 min
idx_batch: 4500
86.2 min
idx_batch: 5000
95.8 min
idx_batch: 5500
105.3 min
idx_batch: 6000
114.9 min
idx_batch: 6500
124.4 min
idx_batch: 7000
134.0 min
idx_batch: 7500
143.6 min
idx_batch: 8000
153.1 min
idx_batch: 8500
162.7 min
idx_batch: 9000
172.3 min
idx_batch: 9500
181.8 min
idx_batch: 10000
191.4 min
idx_batch: 10500
200.9 min
idx_batch: 11000
210.5 min
idx_batch: 11500
220.0 min
idx_batch: 12000
229.6 min
idx_batch: 12500
239.1 min
idx_batch: 13000
248.7 min
idx_batch: 13500
258.2 min
idx_batch: 14000
267.8 min
idx_batch: 14500
277.3 min
idx_batch: 15000
286.9 min
idx_batch: 15500
296.4 min
idx_batch: 16000
306.0 min
idx_batch: 16500
315.5 min
idx_batch: 17000
325.1 min
idx_batch: 17500
334.7 min
idx_batch: 18000
344.2 min
idx_batch: 18500
353.8 min
idx_batch: 

In [ ]:
print('End calculating of the train')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/deep_esn_lorenz_320/log_deep_esn_train_shuffle.npy', log_deep_esn_train_shuffle)

In [ ]:
time_init = time.time()
SIZE_VALID = len(input_valid[:])
log_deep_esn_valid = np.zeros((SIZE_LEAK, SIZE_VALID, SIZE_RESERVOIR*DEPTH_ESN))

In [ ]:
for idx_batch in range(SIZE_VALID):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):    
        input_esn[:, 0] = input_valid[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            deep_esn[idx_leak][0].activate(input_esn[:, :])
            for idx_depth in range(1, DEPTH_ESN):
                deep_esn[idx_leak][idx_depth].activate( \
                    deep_esn[idx_leak][idx_depth-1].state_reservoir[:])

    for idx_leak in range(SIZE_LEAK):
        for idx_depth in range(DEPTH_ESN):
            log_deep_esn_train_shuffle \
                [idx_leak, idx_batch, idx_depth*SIZE_RESERVOIR:(idx_depth+1)*SIZE_RESERVOIR]  \
                = deep_esn[idx_leak][idx_depth].state_reservoir[:]
            # reset reservoir state among time steps
            deep_esn[idx_leak][idx_depth].reset_reservoir_state()

In [ ]:
print('End calculating of the validation')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/deep_esn_lorenz_320/log_deep_esn_valid.npy', log_deep_esn_valid)

In [ ]:
time_init = time.time()
SIZE_TEST = len(input_test[:])
log_deep_esn_test = np.zeros((SIZE_LEAK, SIZE_TEST, SIZE_RESERVOIR*DEPTH_ESN))

In [ ]:
for idx_batch in range(SIZE_TEST):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):    
        input_esn[:, 0] = input_test[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            deep_esn[idx_leak][0].activate(input_esn[:, :])
            for idx_depth in range(1, DEPTH_ESN):
                deep_esn[idx_leak][idx_depth].activate( \
                    deep_esn[idx_leak][idx_depth-1].state_reservoir[:])

    for idx_leak in range(SIZE_LEAK):
        for idx_depth in range(DEPTH_ESN):
            log_deep_esn_train_shuffle \
                [idx_leak, idx_batch, idx_depth*SIZE_RESERVOIR:(idx_depth+1)*SIZE_RESERVOIR]  \
                = deep_esn[idx_leak][idx_depth].state_reservoir[:]
            # reset reservoir state among time steps
            deep_esn[idx_leak][idx_depth].reset_reservoir_state()

In [ ]:
print('End calculating of the test')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/deep_esn_lorenz_320/log_deep_esn_test.npy', log_deep_esn_test)